<a href="https://colab.research.google.com/github/lail-lei/tf-idf/blob/main/tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [206]:
# first, import our libraries 
import pandas as pd

# need for importing text files from github
import requests

# Using regex
import re

In [207]:
# simple, sample documents
documentA = requests.get("https://raw.githubusercontent.com/lail-lei/tf-idf/main/hip_to_be_square.txt").text
documentB = requests.get("https://raw.githubusercontent.com/lail-lei/tf-idf/main/still_alive.txt").text
documentC = requests.get("https://raw.githubusercontent.com/lail-lei/tf-idf/main/wonderful_world_sam_cooke.txt").text

In [208]:
# helper function to process source document before indexing 
def tokenize (src):
  # similar to the standard tokenizer in elasticsearch, this function 
  # lowercases the document, removes punctutaion, and breaks words up on whitespaces 
  # stemming and synonyms would go here 
  return re.sub(r'[^\w\s]', '', src.lower()).split()



class document ():

  document = None
  tokenized_document = None
  inverted_index = None 
  term_frequency_index = None

  # helper function to create inverted index
  def invert (self):
    index = {}
    # populate dictionary with frequency of specific document's words
    for word in self.tokenized_document:
      frequency = index.get(word, 0)
      index[word] = frequency + 1
    return index

  # helper function to create term frequency index
  def tf (self):
    index = {}
    totalNumberOfWords = len(self.tokenized_document)
    for word, count in self.inverted_index.items():
      index[word] = count / float(totalNumberOfWords)
    self.term_frequency_index = index
    
  # constructor
  def __init__(self, document):
    self.document = document
    self.tokenized_document = tokenize(document)
    self.inverted_index = self.invert()
    self.tf()
  

class searchEngine ():

  # a list of all documents in search engine 
  documets = None

  # an index containing the log of the number of documents 
  # divided by the number of documents that contain each specific word
  inverse_document_frequency_index = None

  # a data frame containing the TF-IDF scores for all the words in the engine
  data_frame = None

  # constructor
  def __init__(self):
    self.documents = []

  # compute the idf score across all documents in the corpus 
  def computeIDF(self):
    import math
    N = len(self.documents)
    index = {}
    # store the frequencies of all words across all documents (global frequency)
    for document in self.documents:
        for word, frequency in document.term_frequency_index.items():
            # if word occurs
            if frequency > 0:
              # get previous count if exists
              global_frequency = index.get(word, 0)
              # increment count 
              index[word] = global_frequency + 1
    for word, global_frequency in index.items():
        # perform log of  (#documents/over the global frequency)
        index[word] = math.log(N / float(global_frequency))
    self.inverse_document_frequency_index = index;

  # compute the tfidf score for all words in a document 
  def computeTFIDF(self, document):
    index = {}
    for word, frequency in document.term_frequency_index.items():
        # tf / idf 
        index[word] = frequency * self.inverse_document_frequency_index[word]
    return index

  # store the tfdif values for all documents in the corpus
  def createDataFrame (self):
    data = []
    for document in self.documents:
      index = self.computeTFIDF(document)
      data.append(index)
    self.data_frame = pd.DataFrame(data=data)

  # adds document to search engine
  def index (self, document): 
    # add document to documents list
    self.documents.append(document)
    # compute new idf (optimize later)
    self.computeIDF()
    # create data frame
    self.createDataFrame()

  # look up the tfidf score
  def lookup_tfidf (self, keyword):
    # return empty list if keyword doesn't exist in any document
    if keyword not in self.data_frame.columns:
      return []
    # get the rows (documents) in which the keyword exists
    rows = self.data_frame.index[self.data_frame[keyword] > 0].tolist()
    data = []
    # look up the scores for each row 
    for row in rows:
      score = self.data_frame.iloc[row][keyword]
      data.append({"id": row, "document": self.documents[row].document, "score": score})
    return data
    
  # combine duplictes/sum scores, and order according to relevancy 
  def process_results (self, lookup_list):
    results = {}
    # combine duplicates
    for item in lookup_list:
      if str(item["id"]) not in results:
        results[str(item["id"])] = item 
      else:
        results[str(item["id"])]["score"] += item["score"]
    results = results.values()
    # order in descending order 
    return sorted(results, key=lambda x: x["score"], reverse=True)
         

  # search! 
  def search (self, keyword):
    # process keyword into tokens 
    search_tokens = tokenize(keyword)
    # query the data frame
    results = []
    for token in search_tokens:
     lookup_list = self.lookup_tfidf(token)
     for item in lookup_list:
       results.append(item)
    return self.process_results(results)
  


In [209]:
# create sample engine
songs = searchEngine()

hip = document(documentA)
still = document(documentB)
world = document(documentC)

songs.index(hip)
songs.index(still)
songs.index(world)


In [210]:
songs.search("aperture science")

[{'document': "This was a triumph\nI'm making a note here\nHuge success\nIt's hard to overstate my satisfaction\nAperture Science\nWe do what we must because we can\nFor the good of all of us\nExcept the ones who are dead\nBut there's no sense crying over every mistake\nYou just keep on trying 'til you run out of cake\nAnd the science gets done and you make a neat gun\nFor the people who are still alive\nI'm not even angry\nI'm being so sincere right now\nEven though you broke my heart and killed me\nTore me to pieces\nAnd threw every piece into a fire\nAs they burned it hurt because\nI was so happy for you\nNow these points of data make a beautiful line\nAnd we're out of beta, we're releasing on time\nSo I'm glad I got burned\nThink of all the things we learned\nFor the people who are still alive\nSo go ahead and leave me\nI think I prefer to stay inside\nMaybe you'll find someone else to help you\nMaybe Black Mesa\nThat was a joke, haha, fat chance\nAnyway, this cake is great\nIt's s

In [211]:
songs.search("science book")

[{'document': "Don't know much about history\nDon't know much biology\nDon't know much about a science book\nDon't know much about the French I took\nBut I do know that I love you\nAnd I know that if you love me, too\nWhat a wonderful world this would be\nDon't know much about geography\nDon't know much trigonometry\nDon't know much about algebra\nDon't know what a slide rule is for\nBut I do know one and one is two\nAnd if this one could be with you\nWhat a wonderful world this would be\nNow, I don't claim to be an A student\nBut I'm trying to be\nFor maybe by being an A student, baby\nI can win your love for me\nDon't know much about history\nDon't know much biology\nDon't know much about a science book\nDon't know much about the French I took\nBut I do know that I love you\nAnd I know that if you love me, too\nWhat a wonderful world this would be\nLatatatatata ah\nHistory (Mmmm)\nBiology (Well a-tatatatata)\nScience book (Mmmm)\nFrench I took, yeah\nBut I do know that I love you\nAn

In [212]:
songs.search("square book")

[{'document': "I used to be a renegade\nI used to fool around\nBut I couldn't take the punishment\nAnd had to settle down\nNow I'm playing it real straight\nAnd yes, I cut my hair\nYou might think I'm crazy\nBut I don't even care\nBecause I can tell what's going on\nIt's hip to be square\nIt's hip to be square\nI like my bands in business suits\nI watch them on TV\nI'm working out most every day\nAnd watchin' what I eat\nThey tell me that it's good for me\nBut I don't even care\nI know that it's crazy\nI know that it's nowhere\nBut there is no denying that\nIt's hip to be square\nIt's hip to be square\nIt's hip to be square\nSo hip to be square\nIt's not too hard to figure out\nYou see it every day\nAnd those that were the farthest out\nHave gone the other way\nYou see them on the freeway\nIt don't look like a lot of fun\nBut don't you try to fight it\nAn idea whose time has come\nDon't tell me that I'm crazy\nDon't tell me I'm nowhere\nTake it from me\nIt's hip to be square\nIt's hip 

In [213]:
songs.search("hair")

[{'document': "I used to be a renegade\nI used to fool around\nBut I couldn't take the punishment\nAnd had to settle down\nNow I'm playing it real straight\nAnd yes, I cut my hair\nYou might think I'm crazy\nBut I don't even care\nBecause I can tell what's going on\nIt's hip to be square\nIt's hip to be square\nI like my bands in business suits\nI watch them on TV\nI'm working out most every day\nAnd watchin' what I eat\nThey tell me that it's good for me\nBut I don't even care\nI know that it's crazy\nI know that it's nowhere\nBut there is no denying that\nIt's hip to be square\nIt's hip to be square\nIt's hip to be square\nSo hip to be square\nIt's not too hard to figure out\nYou see it every day\nAnd those that were the farthest out\nHave gone the other way\nYou see them on the freeway\nIt don't look like a lot of fun\nBut don't you try to fight it\nAn idea whose time has come\nDon't tell me that I'm crazy\nDon't tell me I'm nowhere\nTake it from me\nIt's hip to be square\nIt's hip 

In [214]:
# let's import another sample document
documentD = requests.get("https://raw.githubusercontent.com/lail-lei/tf-idf/main/cooking_by_the_book.txt").text

In [215]:
# now let's create a new search engine document and index it 
cook = document(documentD)
songs.index(cook)

In [216]:
songs.search("cooking")

[{'document': "I'll pile on the candy\nIt's such a pretty sight\nIt makes the food taste a dandy\nBut my tummy hurts all night\nI'll put in some ingredients\nBut keep the rest for me\nI'm not just disobedient\nI'm careful can't you see\n\nIt's a piece of cake to bake a pretty cake\nIf you're way is hazy\nYou gotta do the cooking by the book\nYou know you can't be lazy\nNever use a messy recipe\nYour cake may end up crazy\nIf you do the cooking by the book\nThen you'll have a cake\nWe gotta have it made\nYou know that I love cake\nFinally it's time to make a cake\n\nMaking food is just like science\nWith tools that blend and baste\nAnd every fun appliance gives the food a different taste\n\nIt's a piece of cake to bake a pretty cake\nIf you're way is hazy\nYou gotta do the cooking by the book\nYou know you can't be lazy\nNever use a messy recipe\nYour cake may end up crazy\nIf you do the cooking by the book\nThen you'll have a cake\nWe gotta have it made\nYou know that I love cake\nFina

In [217]:
songs.search("cake")

[{'document': "I'll pile on the candy\nIt's such a pretty sight\nIt makes the food taste a dandy\nBut my tummy hurts all night\nI'll put in some ingredients\nBut keep the rest for me\nI'm not just disobedient\nI'm careful can't you see\n\nIt's a piece of cake to bake a pretty cake\nIf you're way is hazy\nYou gotta do the cooking by the book\nYou know you can't be lazy\nNever use a messy recipe\nYour cake may end up crazy\nIf you do the cooking by the book\nThen you'll have a cake\nWe gotta have it made\nYou know that I love cake\nFinally it's time to make a cake\n\nMaking food is just like science\nWith tools that blend and baste\nAnd every fun appliance gives the food a different taste\n\nIt's a piece of cake to bake a pretty cake\nIf you're way is hazy\nYou gotta do the cooking by the book\nYou know you can't be lazy\nNever use a messy recipe\nYour cake may end up crazy\nIf you do the cooking by the book\nThen you'll have a cake\nWe gotta have it made\nYou know that I love cake\nFina

In [218]:
songs.search("all")

[{'document': "This was a triumph\nI'm making a note here\nHuge success\nIt's hard to overstate my satisfaction\nAperture Science\nWe do what we must because we can\nFor the good of all of us\nExcept the ones who are dead\nBut there's no sense crying over every mistake\nYou just keep on trying 'til you run out of cake\nAnd the science gets done and you make a neat gun\nFor the people who are still alive\nI'm not even angry\nI'm being so sincere right now\nEven though you broke my heart and killed me\nTore me to pieces\nAnd threw every piece into a fire\nAs they burned it hurt because\nI was so happy for you\nNow these points of data make a beautiful line\nAnd we're out of beta, we're releasing on time\nSo I'm glad I got burned\nThink of all the things we learned\nFor the people who are still alive\nSo go ahead and leave me\nI think I prefer to stay inside\nMaybe you'll find someone else to help you\nMaybe Black Mesa\nThat was a joke, haha, fat chance\nAnyway, this cake is great\nIt's s